In [1]:
import torch

from rqvae.rqvae_data import get_data

df = get_data()

In [2]:
embs = torch.stack(df["embeddings"].tolist())

In [3]:
import json
from utils import DEVICE
from models.base import BaseModel

config = json.load(open("../configs/train/tiger_train_config.json"))

batch_proc_config = config['dataloader']['train']['batch_processor']

rqvae_train_config = json.load(open(batch_proc_config['rqvae_train_config_path']))
rq_vae_config = rqvae_train_config['model']
rq_vae_config['should_init_codebooks'] = False

rqvae_model = BaseModel.create_from_config(rq_vae_config).to(DEVICE)

rqvae_model.load_state_dict(torch.load(batch_proc_config['rqvae_checkpoint_path'], weights_only=True))
rqvae_model.eval()

ids = df.index.tolist()

embs_dict = {"ids": torch.tensor(ids).to(DEVICE), "embeddings": embs.to(DEVICE)}

semantic_ids = list(rqvae_model.forward(embs_dict))

In [21]:
items_with_tuples

[('7806397051',
  'WAWO 15 Color Professionl Makeup Eyeshadow Camouflage Facial Concealer Neutral Palette',
  (13, 66, 216, 169, 0)),
 ('9759091062', 'Xtreme Brite Brightening Gel 1oz.', (132, 120, 59, 169, 0)),
 ('9788072216',
  'Prada Candy By Prada Eau De Parfum Spray 1.7 Oz For Women',
  (250, 134, 192, 152, 0)),
 ('9790790961',
  'Versace Bright Crystal Eau de Toilette Spray for Women, 3 Ounce',
  (63, 173, 70, 37, 0)),
 ('9790794231', 'Stella McCartney Stella', (58, 80, 40, 27, 0)),
 ('B00004TMFE',
  'Avalon Biotin B-Complex Thickening Conditioner, 14 Ounce',
  (38, 178, 162, 34, 0)),
 ('B00004TUBL',
  'Better Living Classic Two Chamber Dispenser, White',
  (226, 59, 24, 116, 0)),
 ('B00004TUBV',
  'Better Living The Ulti-Mate Dispenser',
  (174, 189, 124, 99, 0)),
 ('B00004U9UY',
  'Crabtree &amp; Evelyn 2792 Gardeners Hand Therapy (100ml, 3.4 oz)',
  (38, 145, 226, 206, 0)),
 ('B00004U9V2',
  "Crabtree &amp; Evelyn - Gardener's Ultra-Moisturising Hand Therapy Pump - 250g/8.8 OZ

In [4]:
from rqvae.collisions import dedup

items_with_tuples = list(zip(df["asin"], df["title"].fillna("unknown"), semantic_ids))
items_with_tuples = dedup(items_with_tuples)

assert len(df) == len(set(item[-1] for item in items_with_tuples))

In [5]:
from trie import Trie

_trie = Trie()

for (id, tuple) in zip(df.index, semantic_ids):
    _trie.insert(tuple, id) # todo handle collisions, not overwrite

In [19]:
import pickle

with open("../data/Beauty/trie.pkl", 'wb') as f:
    pickle.dump(_trie, f)

In [ ]:
from rqvae.rqvae_data import search_similar_items


for i in range(5):
    sim = search_similar_items(items_with_tuples, (i,), 5)
    if len(sim) == 0:
        continue
    print(i)
    for asin, item, clust_tuple in sim:
        # if 'shampoo' in item.lower():
        print(f"{item=} {clust_tuple=}")

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt


plt.hist(Counter(item[-1][:-1] for item in items_with_tuples).values())
plt.show()

In [18]:
# # raw full ids
# full_duplicates = Counter(item[-1][:-1] for item in items_with_tuples).items()
# duplicated = [(semantic_id, amount) for (semantic_id, amount) in full_duplicates if amount > 1]
# duplicated

In [ ]:
# collison counters - (how many item have same full semantic id): amount of such sets
vals = Counter(item[-1][:-1] for item in items_with_tuples).values()
Counter(vals)

In [ ]:
# dedup idxes
Counter(item[-1][4] for item in items_with_tuples)

In [11]:
# from sklearn import preprocessing

# labels = df['asin']

# le = preprocessing.LabelEncoder()
# targets = le.fit_transform(labels)

# df['asin_numeric'] = targets

# torch.save(df, './all_data.pt')

In [ ]:
import torch
from utils import create_masked_tensor


embeddings = torch.rand((11, 2))
print(embeddings)

lengths = torch.tensor([3, 1, 2, 5])

padded_embeddings, mask = create_masked_tensor(embeddings, lengths)

In [ ]:
padded_embeddings

In [ ]:
mask